# 複素ANNによる非線形歪補償
複素数を入力とする3層ANNによる補償

In [1]:
#import
import sys
import os
import time
import datetime
import math

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
from tqdm.notebook import tqdm

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torch.utils.data as data
import torchvision
from torchvision import models, transforms

sys.path.append('../')
from pyopt.util import save_pickle, load_pickle

# 1. Preprocessing

## 1.1 データの整形

In [2]:
def data_shaping(input_signal, signal, max_tap, tap):
    """
    input_signal: 伝送前の信号
    signal: 伝送後の信号
    max_tap: 最大の同時入力シンボル数
    tap: 同時入力シンボル数
    
    signal = [x_0, x_1, ... , x_(n-1)]
      |
      |
      v
    x = [[x_0, x_1, ... , x_tap-1],
            [x_1, x_2, ..., x_tap],
                   .
                   .
                   .
            [x_(n-tap), x_(n-tap+1), ..., x(n-1)]]
      |
      |
      v
    x = [[i_0, q_0, i_1, q_1, ... , i_(tap-1), q_(tap-1)],
            [i_1, q_1, i_2, q_2, ... , i_tap, q_tap],
                   .
                   .
                   .
            [i_(n-tap), q_(n-tap), i_(n-tap+1), q_(n-tap+1), ..., i_(n-1), q_(n-1)]] (batch, input_dim) input_dim = tap * 2
    
    y  (batch, output_dim) output_dim = 2
    """
    
    x = np.zeros((len(input_signal) - (max_tap - 1), tap, 2), dtype=float)
    y = np.zeros((len(input_signal) - (max_tap - 1), 2), dtype=float)
    for i, j in enumerate(np.arange(max_tap // 2, len(input_signal) - max_tap // 2)):
        x[i, :, 0] = signal[j - tap // 2: j + tap // 2 + 1].real
        x[i, :, 1] = signal[j - tap // 2: j + tap // 2 + 1].imag
        y[i, 0] = input_signal[j].real
        y[i, 1] = input_signal[j].imag
    return x, y

In [22]:
#動作確認
tap = 1
max_tap = 51

df_dir = '../data/input/prbs.csv'
df = pd.read_csv(df_dir, index_col=0)  # dataframe読み込み
condition = (df['N']==13) & (df['itr']==1) & (df['form']=='RZ16QAM') & (df['n']==32) & (df['equalize']==False) & (df['baudrate']==28) & (df['PdBm']==1)
sgnl = load_pickle(df[condition].iloc[0]['data_path'])  # dataframeから条件と合う行を取得し,pickleの保存先(data_path)にアクセス
lc = sgnl.linear_compensation(500, sgnl.signal['x_500'])
x, y = data_shaping(sgnl.signal['x_0'][16::32], lc[16::32], max_tap, tap)  # ANNに入力できるようにデータを整形

print('x size: ', x.shape)
print('y size: ', y.shape)
print(x[0])
print(y[0])

x size:  (1998, 1, 2)
y size:  (1998, 2)
[[-53019.95429361  53670.26520552]]
[-70474.95606832  23491.65202277]


In [29]:
x_tensor = torch.Tensor(x)
x_complex = torch.view_as_complex(x_tensor)
print(x_complex)

tensor([[ 51598.7578-53403.3672j, -32145.8477+96508.4375j,
         -30606.3750+95283.5859j,  ...,
         -93925.5391-34256.1562j, -30446.5664-13048.7129j,
           9693.4248-29001.5801j],
        [-32145.8477+96508.4375j, -30606.3750+95283.5859j,
          32503.6094+13910.5205j,  ...,
         -30446.5664-13048.7129j,   9693.4248-29001.5801j,
          53869.6953+53203.7969j],
        [-30606.3750+95283.5859j,  32503.6094+13910.5205j,
           8248.6230+73391.8125j,  ...,
           9693.4248-29001.5801j,  53869.6953+53203.7969j,
         -74834.9922+12591.7324j],
        ...,
        [-57464.2109-54496.4727j, -71785.1406+16135.0635j,
         -37748.6445-11243.9277j,  ...,
         -10821.5039-70501.7188j, -72369.8047+16814.8672j,
         -10232.1523+33094.5391j],
        [-71785.1406+16135.0635j, -37748.6445-11243.9277j,
          96017.2891+27587.5176j,  ...,
         -72369.8047+16814.8672j, -10232.1523+33094.5391j,
         -12742.2695-76211.6172j],
        [-37748.6445-1

## 1.2 平均,標準偏差の計算

In [23]:
mean = np.mean(x)
std = np.std(x)

print('mean: ', mean)
print('std: ', std)

mean:  1138.625923374995
std:  52116.95097240129


# 2. Dataset定義

In [3]:
class Dataset(data.Dataset):
    def __init__(self, x, y, mean, std):
        self.x, self.y, self.mean, self.std = x, y, mean, std
    
    def __len__(self):
        return len(self.x)
    
    def __getitem__(self, index):
        x = self.x[index]
        y = self.y[index]
        
        x = (x - self.mean) / self.std
        y = (y - self.mean) / self.std
        
        x = torch.Tensor(x)
        y = torch.Tensor(y)
        
        x_i = x[:, 0]
        x_q = x[:, 1]
        y_i = y[0]
        y_q = y[1]
        return x_i, x_q, y_i, y_q

In [24]:
#動作確認
train_dataset = Dataset(x=x, y=y, mean=mean, std=std)

index = 0
x_i, x_q, y_i, y_q = train_dataset.__getitem__(index)
x_array = x_i.detach().numpy()

print('mean: ', np.mean(x_array))
print('std: ', np.std(x_array))
print(x_i)
print(y_i)

mean:  -1.039174
std:  0.0
tensor([-1.0392])
tensor(-1.3741)


In [25]:
batch_size = 100

train_dataloader = data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

dataloaders_dict = {'train': train_dataloader}

# 3. Model定義

In [4]:
def zrelu(x_i, x_q):
    x_i, x_q = x_i.clone(), x_q.clone()
    for i in range(x_i.shape[0]):
        for j in range(x_i.shape[1]):
            if x_i[i, j] < 0 or x_q[i, j] < 0:
                x_i[i, j] = 0
                x_q[i, j] = 0
    return x_i, x_q

In [5]:
def crelu(x_i, x_q):
    x_i, x_q = x_i.clone(), x_q.clone()
    x_i, x_q = F.relu(x_i), F.relu(x_q)
    return x_i, x_q

In [6]:
class ComplexLinear(nn.Module):
    def __init__(self, in_features, out_features):
        super().__init__()
        self.in_features = in_features
        self.out_features = out_features
        
        # 重み定義 Xavierの初期値
        k = 1 / in_features
        weight_i = torch.empty(out_features, in_features).uniform_(-math.sqrt(k), math.sqrt(k))
        self.weight_i = nn.Parameter(weight_i)
        weight_q = torch.empty(out_features, in_features).uniform_(-math.sqrt(k), math.sqrt(k))
        self.weight_q = nn.Parameter(weight_q)
        
        bias_i = torch.empty(out_features).uniform_(-k, k)
        self.bias_i = nn.Parameter(bias_i)
        bias_q = torch.empty(out_features).uniform_(-k, k)
        self.bias_q = nn.Parameter(bias_q)
        
    def forward(self, x_i, x_q):
        i = nn.functional.linear(x_i, self.weight_i, self.bias_i)
        q = nn.functional.linear(x_q, self.weight_q, self.bias_q)
        return i - q, i + q

In [20]:
class ComplexANN(nn.Module):
    def __init__(self, input_dim, output_dim, hidden_neuron):
        super().__init__()
        self.fc1 = ComplexLinear(input_dim, hidden_neuron)
        self.fc2 = ComplexLinear(hidden_neuron, output_dim)
    
    def forward(self, x_i, x_q):
        x_i, x_q = self.fc1(x_i, x_q)
        x_i, x_q = crelu(x_i, x_q)
        x_i, x_q = self.fc2(x_i, x_q)
        return x_i, x_q

In [17]:
#動作確認
hidden_neuron = 300
device = torch.device('cpu') # 'cuda' if torch.cuda.is_available() else 
print('Device available now:', device)

model = ComplexANN(input_dim=tap, output_dim=1, hidden_neuron=hidden_neuron).to(device)
for x_i, x_q, y_i, y_q in train_dataloader:
    out_i, out_q = model(x_i, x_q)
    print(out_i)
    print(out_i.shape)
    break

Device available now: cpu
tensor([[-0.2868],
        [-0.2514],
        [ 0.0588],
        [ 0.2231],
        [-0.4915],
        [-0.3835],
        [ 0.1541],
        [ 0.0466],
        [ 0.3044],
        [ 0.2885],
        [-0.2808],
        [-0.3512],
        [-0.5910],
        [ 0.0301],
        [ 0.6442],
        [ 0.4462],
        [ 0.1734],
        [-0.6990],
        [ 0.6004],
        [ 0.1038],
        [ 0.5551],
        [-0.0911],
        [ 0.0028],
        [ 0.0315],
        [ 0.1260],
        [ 0.0396],
        [ 0.0013],
        [ 0.4779],
        [-0.2087],
        [ 0.3371],
        [-0.6761],
        [ 0.5659],
        [ 0.5817],
        [-0.0913],
        [ 0.1441],
        [ 0.0471],
        [-0.4767],
        [-0.0612],
        [-0.2799],
        [-0.1568],
        [ 0.1588],
        [-0.1137],
        [ 0.3336],
        [-0.2513],
        [ 0.7978],
        [ 0.0233],
        [-0.1835],
        [ 0.6011],
        [ 0.0310],
        [ 0.0768],
        [-0.2429],
     

# 4. train定義

In [8]:
def evm_score(x_i, x_q, y_i, y_q):
    tmp = 0
    for i in range(len(x_i)):
        tmp += ((x_i[i] - y_i[i]) ** 2 + (x_q[i] - y_q[i]) ** 2) / (y_i[i] ** 2 + y_q[i] ** 2)
    evm = torch.sqrt(tmp / len(x_i))
    return evm

In [34]:
def train_model(device, model, dataloaders_dict, criterion, optimizer, epochs, epochs_section=None):
    for epoch in range(epochs):
        if epochs_section is not None:
            epoch += epochs_section[0]
            end_epoch = epochs_section[1]
        else:
            end_epoch = epochs
        
        start_time = time.time()
        
        for phase in dataloaders_dict.keys():
            if phase == 'train':
                model.train()
            else:
                model.eval()
            
            epoch_loss = 0.0
            epoch_evms = 0.0
            
            for x_i, x_q, y_i, y_q in dataloaders_dict[phase]:
                x_i, x_q = x_i.to(device), x_q.to(device)
                print(x_i.shape)
                y_i, y_q = y_i.to(device), y_q.to(device)
                
                optimizer.zero_grad()
                
                with torch.set_grad_enabled(phase == 'train'):
                    out_i, out_q = model(x_i, x_q)
                    loss = evm_score(out_i, out_q, y_i, y_q)
                    
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()
                    
                    epoch_loss += loss.item() * x_i.size(0)
                    epoch_evms += (evm_score(out_i, out_q, y_i, y_q)) ** 2 * x_i.size(0)
            
            epoch_loss = epoch_loss / len(dataloaders_dict[phase].dataset)
            epoch_evm = torch.sqrt(epoch_evms / len(dataloaders_dict[phase].dataset)) * 100
            
            duration = str(datetime.timedelta(seconds=time.time() - start_time))[:7]
            print('{} | Epoch: {}/{} | {} Loss: {:.4} | EVM: {:.4}'.format(duration, epoch + 1, end_epoch, phase, epoch_loss, epoch_evm[0]))
    return model

In [35]:
#動作確認
epochs = 5
lr = 0.001

criterion = nn.MSELoss()
optimizer = optim.Adam(params=model.parameters(), lr=lr)

train_model(device=device, model=model, dataloaders_dict=dataloaders_dict, criterion=criterion, optimizer=optimizer, epochs=epochs);

torch.Size([100, 201])
torch.Size([100, 201])
torch.Size([100, 201])
torch.Size([100, 201])
torch.Size([100, 201])
torch.Size([100, 201])
torch.Size([100, 201])
torch.Size([100, 201])
torch.Size([100, 201])
torch.Size([100, 201])
torch.Size([100, 201])
torch.Size([100, 201])
torch.Size([100, 201])
torch.Size([100, 201])
torch.Size([100, 201])
torch.Size([48, 201])
0:00:00 | Epoch: 1/5 | train Loss: 0.06023 | EVM: 6.171
torch.Size([100, 201])
torch.Size([100, 201])
torch.Size([100, 201])
torch.Size([100, 201])
torch.Size([100, 201])
torch.Size([100, 201])
torch.Size([100, 201])
torch.Size([100, 201])
torch.Size([100, 201])
torch.Size([100, 201])
torch.Size([100, 201])
torch.Size([100, 201])
torch.Size([100, 201])
torch.Size([100, 201])
torch.Size([100, 201])
torch.Size([100, 201])
torch.Size([100, 201])
torch.Size([100, 201])
torch.Size([100, 201])
torch.Size([100, 201])
torch.Size([100, 201])
torch.Size([100, 201])
torch.Size([100, 201])
torch.Size([100, 201])
torch.Size([100, 201])
to

torch.Size([100, 201])
torch.Size([48, 201])
0:00:00 | Epoch: 2/5 | train Loss: 0.08536 | EVM: 8.608
torch.Size([100, 201])
torch.Size([100, 201])
torch.Size([100, 201])
torch.Size([100, 201])
torch.Size([100, 201])
torch.Size([100, 201])
torch.Size([100, 201])
torch.Size([100, 201])
torch.Size([100, 201])
torch.Size([100, 201])
torch.Size([100, 201])
torch.Size([100, 201])
torch.Size([100, 201])
torch.Size([100, 201])
torch.Size([100, 201])
torch.Size([100, 201])
torch.Size([100, 201])
torch.Size([100, 201])
torch.Size([100, 201])
torch.Size([100, 201])
torch.Size([100, 201])
torch.Size([100, 201])
torch.Size([100, 201])
torch.Size([100, 201])
torch.Size([100, 201])
torch.Size([100, 201])
torch.Size([100, 201])
torch.Size([100, 201])
torch.Size([100, 201])
torch.Size([100, 201])
torch.Size([100, 201])
torch.Size([100, 201])
torch.Size([100, 201])
torch.Size([100, 201])
torch.Size([100, 201])
torch.Size([100, 201])
torch.Size([100, 201])
torch.Size([100, 201])
torch.Size([100, 201])
to

KeyboardInterrupt: 

In [27]:
#結果を保存しない
tap = 201
max_tap = 501
batch_size = 100
hidden_neuron = 300
epochs = 500
lr = 0.001

device = torch.device('cpu') # 'cuda' if torch.cuda.is_available() else 
print('Device available now:', device)

df_dir = '../data/input/'
df0 = pd.read_csv(df_dir+'prbs.csv', index_col=0)

condition0 = (df0['N']==13) & (df0['itr']==1) & (df0['form']=='RZ16QAM') & (df0['n']==32) & (df0['equalize']==False) & (df0['baudrate']==28) & (df0['PdBm']==1)
sgnl0 = load_pickle(df0[condition0].iloc[0]['data_path'])
lc0 = sgnl0.linear_compensation(2500, sgnl0.signal['x_2500'])
x0, y0 = data_shaping(sgnl0.signal['x_0'][16::32], lc0[16::32], max_tap, tap)

condition1 = (df0['N']==17) & (df0['itr']==1) & (df0['form']=='RZ16QAM') & (df0['n']==32) & (df0['equalize']==False) & (df0['baudrate']==28) & (df0['PdBm']==1)
sgnl1 = load_pickle(df0[condition1].iloc[0]['data_path'])
lc1 = sgnl1.linear_compensation(2500, sgnl1.signal['x_2500'])
x1, y1 = data_shaping(sgnl1.signal['x_0'][16::32], lc1[16::32], max_tap, tap)

mean = np.mean(x0)
std = np.std(x0)

train_dataset = Dataset(x=x0, y=y0, mean=mean, std=std)
val_dataset = Dataset(x=x1, y=y1, mean=mean, std=std)

train_dataloader = data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_dataloader = data.DataLoader(val_dataset, batch_size=batch_size, shuffle=False)
dataloaders_dict = {'train': train_dataloader, 'val': val_dataloader}

model = ComplexANN(input_dim=tap, output_dim=1, hidden_neuron=hidden_neuron).to(device)
criterion = nn.MSELoss()
optimizer = optim.Adam(params=model.parameters(), lr=lr)

model = train_model(device=device, model=model, dataloaders_dict=dataloaders_dict, criterion=criterion, optimizer=optimizer, epochs=epochs)

Device available now: cpu
0:00:01 | Epoch: 1/500 | train Loss: 1.088 | EVM: 108.9
0:00:11 | Epoch: 1/500 | val Loss: 0.9937 | EVM: 99.42
0:00:01 | Epoch: 2/500 | train Loss: 0.7547 | EVM: 75.56
0:00:11 | Epoch: 2/500 | val Loss: 0.8873 | EVM: 88.78
0:00:01 | Epoch: 3/500 | train Loss: 0.5538 | EVM: 55.43
0:00:11 | Epoch: 3/500 | val Loss: 0.7985 | EVM: 79.92
0:00:00 | Epoch: 4/500 | train Loss: 0.3874 | EVM: 38.85
0:00:11 | Epoch: 4/500 | val Loss: 0.7117 | EVM: 71.26
0:00:00 | Epoch: 5/500 | train Loss: 0.3065 | EVM: 30.76
0:00:10 | Epoch: 5/500 | val Loss: 0.6638 | EVM: 66.49
0:00:00 | Epoch: 6/500 | train Loss: 0.4017 | EVM: 40.4
0:00:11 | Epoch: 6/500 | val Loss: 0.643 | EVM: 64.39
0:00:00 | Epoch: 7/500 | train Loss: 0.3858 | EVM: 39.13
0:00:11 | Epoch: 7/500 | val Loss: 0.6146 | EVM: 61.54
0:00:00 | Epoch: 8/500 | train Loss: 0.3322 | EVM: 33.65
0:00:11 | Epoch: 8/500 | val Loss: 0.6024 | EVM: 60.33
0:00:00 | Epoch: 9/500 | train Loss: 0.2902 | EVM: 29.27
0:00:11 | Epoch: 9/500 |

0:00:01 | Epoch: 73/500 | train Loss: 0.09286 | EVM: 9.385
0:00:12 | Epoch: 73/500 | val Loss: 0.3553 | EVM: 35.59
0:00:00 | Epoch: 74/500 | train Loss: 0.09838 | EVM: 10.02
0:00:11 | Epoch: 74/500 | val Loss: 0.3555 | EVM: 35.61
0:00:00 | Epoch: 75/500 | train Loss: 0.1056 | EVM: 10.67
0:00:11 | Epoch: 75/500 | val Loss: 0.352 | EVM: 35.26
0:00:01 | Epoch: 76/500 | train Loss: 0.09235 | EVM: 9.322
0:00:12 | Epoch: 76/500 | val Loss: 0.3534 | EVM: 35.41
0:00:01 | Epoch: 77/500 | train Loss: 0.1021 | EVM: 10.4
0:00:12 | Epoch: 77/500 | val Loss: 0.3505 | EVM: 35.12
0:00:00 | Epoch: 78/500 | train Loss: 0.08869 | EVM: 9.005
0:00:11 | Epoch: 78/500 | val Loss: 0.3496 | EVM: 35.02
0:00:00 | Epoch: 79/500 | train Loss: 0.08887 | EVM: 8.986
0:00:11 | Epoch: 79/500 | val Loss: 0.3525 | EVM: 35.32
0:00:00 | Epoch: 80/500 | train Loss: 0.1001 | EVM: 10.15
0:00:11 | Epoch: 80/500 | val Loss: 0.3511 | EVM: 35.17
0:00:00 | Epoch: 81/500 | train Loss: 0.09684 | EVM: 9.754
0:00:11 | Epoch: 81/500 | 

0:00:00 | Epoch: 144/500 | train Loss: 0.07083 | EVM: 7.134
0:00:12 | Epoch: 144/500 | val Loss: 0.3012 | EVM: 30.18
0:00:00 | Epoch: 145/500 | train Loss: 0.07256 | EVM: 7.393
0:00:11 | Epoch: 145/500 | val Loss: 0.3009 | EVM: 30.15
0:00:00 | Epoch: 146/500 | train Loss: 0.07288 | EVM: 7.328
0:00:11 | Epoch: 146/500 | val Loss: 0.2977 | EVM: 29.83
0:00:00 | Epoch: 147/500 | train Loss: 0.06959 | EVM: 7.077
0:00:11 | Epoch: 147/500 | val Loss: 0.3014 | EVM: 30.21
0:00:00 | Epoch: 148/500 | train Loss: 0.07081 | EVM: 7.213
0:00:11 | Epoch: 148/500 | val Loss: 0.3016 | EVM: 30.22
0:00:00 | Epoch: 149/500 | train Loss: 0.06907 | EVM: 6.998
0:00:12 | Epoch: 149/500 | val Loss: 0.2994 | EVM: 30.01
0:00:00 | Epoch: 150/500 | train Loss: 0.07178 | EVM: 7.315
0:00:11 | Epoch: 150/500 | val Loss: 0.3002 | EVM: 30.08
0:00:00 | Epoch: 151/500 | train Loss: 0.07133 | EVM: 7.23
0:00:12 | Epoch: 151/500 | val Loss: 0.2983 | EVM: 29.89
0:00:00 | Epoch: 152/500 | train Loss: 0.07327 | EVM: 7.368
0:00:

0:00:11 | Epoch: 214/500 | val Loss: 0.2814 | EVM: 28.21
0:00:00 | Epoch: 215/500 | train Loss: 0.06347 | EVM: 6.395
0:00:11 | Epoch: 215/500 | val Loss: 0.274 | EVM: 27.46
0:00:00 | Epoch: 216/500 | train Loss: 0.06171 | EVM: 6.2
0:00:12 | Epoch: 216/500 | val Loss: 0.2743 | EVM: 27.5
0:00:01 | Epoch: 217/500 | train Loss: 0.06248 | EVM: 6.351
0:00:12 | Epoch: 217/500 | val Loss: 0.2754 | EVM: 27.61
0:00:00 | Epoch: 218/500 | train Loss: 0.06309 | EVM: 6.378
0:00:12 | Epoch: 218/500 | val Loss: 0.275 | EVM: 27.57
0:00:01 | Epoch: 219/500 | train Loss: 0.06277 | EVM: 6.331
0:00:12 | Epoch: 219/500 | val Loss: 0.2755 | EVM: 27.62
0:00:01 | Epoch: 220/500 | train Loss: 0.06626 | EVM: 6.693
0:00:11 | Epoch: 220/500 | val Loss: 0.2735 | EVM: 27.41
0:00:00 | Epoch: 221/500 | train Loss: 0.06569 | EVM: 6.651
0:00:11 | Epoch: 221/500 | val Loss: 0.276 | EVM: 27.66
0:00:00 | Epoch: 222/500 | train Loss: 0.06302 | EVM: 6.375
0:00:11 | Epoch: 222/500 | val Loss: 0.273 | EVM: 27.37
0:00:01 | Epoc

0:00:00 | Epoch: 285/500 | train Loss: 0.05754 | EVM: 5.797
0:00:11 | Epoch: 285/500 | val Loss: 0.2649 | EVM: 26.56
0:00:00 | Epoch: 286/500 | train Loss: 0.05664 | EVM: 5.724
0:00:12 | Epoch: 286/500 | val Loss: 0.2614 | EVM: 26.21
0:00:00 | Epoch: 287/500 | train Loss: 0.06019 | EVM: 6.05
0:00:11 | Epoch: 287/500 | val Loss: 0.2623 | EVM: 26.3
0:00:00 | Epoch: 288/500 | train Loss: 0.05852 | EVM: 5.902
0:00:11 | Epoch: 288/500 | val Loss: 0.2651 | EVM: 26.58
0:00:01 | Epoch: 289/500 | train Loss: 0.05639 | EVM: 5.694
0:00:11 | Epoch: 289/500 | val Loss: 0.2644 | EVM: 26.51
0:00:00 | Epoch: 290/500 | train Loss: 0.05486 | EVM: 5.599
0:00:12 | Epoch: 290/500 | val Loss: 0.2593 | EVM: 25.99
0:00:00 | Epoch: 291/500 | train Loss: 0.05543 | EVM: 5.594
0:00:11 | Epoch: 291/500 | val Loss: 0.2655 | EVM: 26.61
0:00:01 | Epoch: 292/500 | train Loss: 0.05549 | EVM: 5.606
0:00:11 | Epoch: 292/500 | val Loss: 0.2642 | EVM: 26.48
0:00:00 | Epoch: 293/500 | train Loss: 0.05971 | EVM: 6.055
0:00:1

0:00:11 | Epoch: 355/500 | val Loss: 0.2571 | EVM: 25.78
0:00:00 | Epoch: 356/500 | train Loss: 0.05655 | EVM: 5.717
0:00:11 | Epoch: 356/500 | val Loss: 0.2569 | EVM: 25.76
0:00:00 | Epoch: 357/500 | train Loss: 0.05157 | EVM: 5.206
0:00:11 | Epoch: 357/500 | val Loss: 0.2586 | EVM: 25.92
0:00:00 | Epoch: 358/500 | train Loss: 0.05101 | EVM: 5.122
0:00:10 | Epoch: 358/500 | val Loss: 0.2565 | EVM: 25.72
0:00:00 | Epoch: 359/500 | train Loss: 0.0592 | EVM: 5.97
0:00:11 | Epoch: 359/500 | val Loss: 0.2573 | EVM: 25.8
0:00:00 | Epoch: 360/500 | train Loss: 0.05758 | EVM: 5.808
0:00:11 | Epoch: 360/500 | val Loss: 0.2574 | EVM: 25.81
0:00:00 | Epoch: 361/500 | train Loss: 0.05317 | EVM: 5.439
0:00:10 | Epoch: 361/500 | val Loss: 0.2567 | EVM: 25.74
0:00:00 | Epoch: 362/500 | train Loss: 0.05305 | EVM: 5.375
0:00:11 | Epoch: 362/500 | val Loss: 0.2563 | EVM: 25.7
0:00:00 | Epoch: 363/500 | train Loss: 0.05087 | EVM: 5.124
0:00:11 | Epoch: 363/500 | val Loss: 0.2571 | EVM: 25.78
0:00:00 | E

0:00:00 | Epoch: 426/500 | train Loss: 0.05639 | EVM: 5.709
0:00:11 | Epoch: 426/500 | val Loss: 0.2503 | EVM: 25.1
0:00:00 | Epoch: 427/500 | train Loss: 0.048 | EVM: 4.865
0:00:11 | Epoch: 427/500 | val Loss: 0.2535 | EVM: 25.42
0:00:00 | Epoch: 428/500 | train Loss: 0.0477 | EVM: 4.822
0:00:11 | Epoch: 428/500 | val Loss: 0.2524 | EVM: 25.31
0:00:00 | Epoch: 429/500 | train Loss: 0.05224 | EVM: 5.282
0:00:11 | Epoch: 429/500 | val Loss: 0.2547 | EVM: 25.54
0:00:00 | Epoch: 430/500 | train Loss: 0.0518 | EVM: 5.206
0:00:10 | Epoch: 430/500 | val Loss: 0.2528 | EVM: 25.34
0:00:00 | Epoch: 431/500 | train Loss: 0.04965 | EVM: 5.008
0:00:11 | Epoch: 431/500 | val Loss: 0.2547 | EVM: 25.54
0:00:00 | Epoch: 432/500 | train Loss: 0.04994 | EVM: 5.033
0:00:11 | Epoch: 432/500 | val Loss: 0.2528 | EVM: 25.35
0:00:00 | Epoch: 433/500 | train Loss: 0.04851 | EVM: 4.933
0:00:11 | Epoch: 433/500 | val Loss: 0.2545 | EVM: 25.52
0:00:00 | Epoch: 434/500 | train Loss: 0.04995 | EVM: 5.039
0:00:11 |

0:00:11 | Epoch: 496/500 | val Loss: 0.2515 | EVM: 25.22
0:00:00 | Epoch: 497/500 | train Loss: 0.05107 | EVM: 5.181
0:00:11 | Epoch: 497/500 | val Loss: 0.2517 | EVM: 25.24
0:00:00 | Epoch: 498/500 | train Loss: 0.04572 | EVM: 4.64
0:00:11 | Epoch: 498/500 | val Loss: 0.2481 | EVM: 24.88
0:00:00 | Epoch: 499/500 | train Loss: 0.04847 | EVM: 4.873
0:00:11 | Epoch: 499/500 | val Loss: 0.2505 | EVM: 25.12
0:00:00 | Epoch: 500/500 | train Loss: 0.04482 | EVM: 4.538
0:00:11 | Epoch: 500/500 | val Loss: 0.2538 | EVM: 25.45


In [51]:
print(train_dataset[0][0].shape)

torch.Size([201])


In [ ]:
#可視化
annc0 = model(train_dataset[:][0]).detach().numpy()
annc0 = annc0 * std + mean
annc0 = annc0[:, 0] + annc0[:, 1] * 1j
y0 = train_dataset[:][1].detach().numpy()
y0 = y0 * std + mean
y0 = y0.astype(np.int32)
y0 = y0[:, 0] + y0[:, 1] * 1j
annc1 = model(val_dataset[:][0]).detach().numpy()
annc1 = annc1 * std + mean
annc1 = annc1[:, 0] + annc1[:, 1] * 1j
y1 = val_dataset[:][1].detach().numpy()
y1 = y1 * std + mean
y1 = y1.astype(np.int32)
y1 = y1[:, 0] + y1[:, 1] * 1j

lim = 110000
cm = plt.get_cmap('rainbow', 16)

seq0 = sgnl0.signal['x_0'][16::32]
seq1 = sgnl1.signal['x_0'][16::32]
symbol, inverse, counts = np.unique(seq0, return_inverse=True, return_counts=True)
symbol_int = symbol.real.astype(np.int32) + symbol.imag.astype(np.int32) * 1j

fig = plt.figure(figsize=(16, 17))
ax1 = fig.add_subplot(2, 2, 1)
ax2 = fig.add_subplot(2, 2, 2)
ax3 = fig.add_subplot(2, 2, 3)
ax4 = fig.add_subplot(2, 2, 4)

for i in range(len(symbol)):
    ax1.plot(lc0[16::32][seq0 == symbol[i]].real, lc0[16::32][seq0 == symbol[i]].imag, '.', color=[cm(i)[0], cm(i)[1], cm(i)[2]], label=str(counts[i]))
for i in range(len(symbol)):
    ax1.plot(seq0[seq0 == symbol[i]].real, seq0[seq0 == symbol[i]].imag, 'o', color=[cm(i)[0], cm(i)[1], cm(i)[2]], markersize=10, markeredgewidth=1, markeredgecolor='black')
ax1.set_title('constellation PRBS13(train) 100%linear comp.')
ax1.set_xlim(-lim, lim)
ax1.set_ylim(-lim, lim)

for i in range(len(symbol)):
    ax2.plot(lc1[16::32][seq1 == symbol[i]].real, lc1[16::32][seq1 == symbol[i]].imag, '.', color=[cm(i)[0], cm(i)[1], cm(i)[2]], label=str(counts[i]))
for i in range(len(symbol)):
    ax2.plot(seq1[seq1 == symbol[i]].real, seq1[seq1 == symbol[i]].imag, 'o', color=[cm(i)[0], cm(i)[1], cm(i)[2]], markersize=10, markeredgewidth=1, markeredgecolor='black')
ax2.set_title('constellation PRBS17(test) 100%linear comp.')
ax2.set_xlim(-lim, lim)
ax2.set_ylim(-lim, lim)

for i in range(len(symbol)):
    ax3.plot(annc0[y0 == symbol_int[i]].real, annc0[y0 == symbol_int[i]].imag, '.', color=[cm(i)[0], cm(i)[1], cm(i)[2]])
for i in range(len(symbol)):
    ax3.plot(y0[y0 == symbol_int[i]].real, y0[y0 == symbol_int[i]].imag, 'o', color=[cm(i)[0], cm(i)[1], cm(i)[2]], markersize=10, markeredgewidth=1, markeredgecolor='black')
ax3.set_title('constellation PRBS13(train) 100%linear+ANN comp.')
ax3.set_xlim(-lim, lim)
ax3.set_ylim(-lim, lim)

for i in range(len(symbol)):
    ax4.plot(annc1[y1 == symbol_int[i]].real, annc1[y1 == symbol_int[i]].imag, '.', color=[cm(i)[0], cm(i)[1], cm(i)[2]])
for i in range(len(symbol)):
    ax4.plot(y1[y1 == symbol_int[i]].real, y1[y1 == symbol_int[i]].imag, 'o', color=[cm(i)[0], cm(i)[1], cm(i)[2]], markersize=10, markeredgewidth=1, markeredgecolor='black')
ax4.set_title('constellation PRBS17(test) 100%linear+ANN comp.')
ax4.set_xlim(-lim, lim)
ax4.set_ylim(-lim, lim);